# Get data via SQLAlchemy

In [25]:
import sqlalchemy, pickle
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy import and_, or_
import MySQLdb, pickle

### Access to MySQL Server

In [26]:
pw = pickle.load(open("../crawler/mysql_pw.pickle", "rb"))

In [27]:
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + pw + "@52.78.44.120/project_rookie")

### ORM

In [28]:
Base = declarative_base()
class Debut(Base):
    __tablename__ = 'debut_album'

    index = Column(Integer, primary_key=True)
    artist = Column(String)
    album = Column(String)
    genre = Column(String)
    single_count = Column(Integer)
    
    def __init__(self, index, artist, album, genre, single_count):
        self.index = index
        self.artist = artist
        self.album = album
        self.genre = genre
        self.single_count = single_count
    
    def __repr__(self):
        return "<index {}, {}, {}, {}, {}>".format(self.index, self.artist, self.album, self.genre, self.single_count)

In [29]:
Base.metadata.create_all(engine)

In [30]:
Session = sessionmaker(bind=engine)
session = Session()

In [31]:
def disp(datas):
    for data in datas:
        print(data)

In [32]:
results = session.query(Debut).all()
results[0].__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x111d925c0>,
 'album': '2Cellos',
 'artist': '2Cellos',
 'genre': 'Cello rock, classical crossover',
 'index': 0,
 'single_count': 3}

### Convert data into dataframe

In [33]:
def orm_to_df(orm):

    df = pd.DataFrame(columns=['artist', 'album', 'genre', 'single_count'])

    for i in orm:

        data = {
            'artist' : i.__dict__['artist'],
            'album' : i.__dict__['album'],
            'genre' : i.__dict__['genre'],
            'single_count' : i.__dict__['single_count'],
        }
        df.loc[len(df)] = data
        
    return df

### Regroup genres into `hiphop`, `rnb`, `soul`, `funk`, `pop`

In [34]:
import pandas as pd

In [35]:
hiphop = session.query(Debut).filter(and_(~Debut.genre.contains(['K-pop']), Debut.genre.contains(["hop"])))
hiphop_df = orm_to_df(hiphop)
print(hiphop_df.shape)
hiphop_df.head()

(294, 4)


,artist,album,genre,single_count
0,Flash Bang Grenada,10 Haters,Alternative hip hop,0
1,Aggro Santos,AggroSantos.com,"Hip hop, dance",3
2,AKA,Altar Ego,"Hip hop, Rap, English, Setswana, isiZulu",4
3,Shlohmo,Bad Vibes,"Electronic, instrumental hip hop",0
4,Glasses Malone,Beach Cruiser,"Gangsta rap, West Coast hip hop",4


In [36]:
rap = session.query(Debut).filter(and_(~Debut.genre.contains(['K-pop']), Debut.genre.contains(["Rap"])))
rap_df = orm_to_df(rap)
print(rap_df.shape)
rap_df.head()

(80, 4)


,artist,album,genre,single_count
0,AKA,Altar Ego,"Hip hop, Rap, English, Setswana, isiZulu",4
1,Glasses Malone,Beach Cruiser,"Gangsta rap, West Coast hip hop",4
2,Cities Aviv,Digital Lows,"Alternative hip hop, cloud rap, experimental h...",0
3,B.G. Knocc Out,Eazy-E's Protege,"G-funk, West Coast hip hop, Gangsta rap",1
4,Jay Rock,Follow Me Home,"West coast hip hop, gangsta rap",2


In [37]:
trap = session.query(Debut).filter(and_(~Debut.genre.contains(['K-pop']), Debut.genre.contains(["trap"])))
trap_df = orm_to_df(trap)
print(trap_df.shape)
trap_df.head()

(29, 4)


,artist,album,genre,single_count
0,Joke,Ateyaba,"Hip hop, french rap, trap, Hardcore Rap, Dirty...",7
1,Dillon Francis,Money Sucks Friends Rule,"Electro house, EDM, synth-pop, trap, moombahton",7
2,Emtee,Avery,"Trap, English, Sesotho, Setswana, isiZulu",5
3,Fetty Wap,Fetty Wap,"Hip hop, trap",4
4,Lil Durk,Remember My Name,"Hip hop, trap, drill",1


In [38]:
hiphop_df = pd.concat([hiphop_df, rap_df, trap_df])
hiphop_df.drop_duplicates(['artist', 'album'], inplace=True)
hiphop_df['genre'] = "hiphop"
hiphop_df

,artist,album,genre,single_count
0,Flash Bang Grenada,10 Haters,hiphop,0
1,Aggro Santos,AggroSantos.com,hiphop,3
2,AKA,Altar Ego,hiphop,4
3,Shlohmo,Bad Vibes,hiphop,0
4,Glasses Malone,Beach Cruiser,hiphop,4
5,Shabazz Palaces,Black Up,hiphop,0
6,Mac Miller,Blue Slide Park,hiphop,4
7,Phonte,Charity Starts at Home,hiphop,0
8,J. Cole,Cole World: The Sideline Story,hiphop,3
9,Cities Aviv,Digital Lows,hiphop,0


In [44]:
rnb = session.query(Debut).filter(and_(~Debut.genre.contains(['K-pop']), Debut.genre.contains(["R&B"])))
rnb_df = orm_to_df(rnb)
rnb_df['genre'] = 'rnb'
print(rnb_df.shape)
rnb_df.head()

(197, 4)


,artist,album,genre,single_count
0,Alexis Jordan,Alexis Jordan,rnb,3
1,Phonte,Charity Starts at Home,rnb,0
2,Joe Jonas,Fastlife,rnb,2
3,Romeo Santos,Formula Vol. 1,rnb,6
4,Alyssa Reid,The Game,rnb,3


In [45]:
soul = session.query(Debut).filter(and_(~Debut.genre.contains(['K-pop']), Debut.genre.contains(["soul"])))
soul_df = orm_to_df(soul)
soul_df['genre'] = 'soul'
print(soul_df.shape)
soul_df.head()

(112, 4)


,artist,album,genre,single_count
0,Andy Grammer,Andy Grammer,soul,3
1,Vintage Trouble,The Bomb Shelter Sessions,soul,3
2,The Horrible Crowes,Elsie,soul,1
3,Jamala,For Every Heart,soul,3
4,Graham Alexander,Graham Alexander,soul,0


In [46]:
funk = session.query(Debut).filter(and_(~Debut.genre.contains(['K-pop']), Debut.genre.contains(["funk"])))
funk_df = orm_to_df(funk)
funk_df['genre'] = 'funk'
print(funk_df.shape)
funk_df.head()

(28, 4)


,artist,album,genre,single_count
0,Ford & Lopatin,Channel Pressure,funk,2
1,B.G. Knocc Out,Eazy-E's Protege,funk,1
2,Graham Alexander,Graham Alexander,funk,0
3,Katie Waissel,Katie Waissel,funk,0
4,Charles Bradley,No Time for Dreaming,funk,2


In [47]:
pop = session.query(Debut).filter(and_(~Debut.genre.contains(['K-pop']), Debut.genre.contains(["pop"])))
pop_df = orm_to_df(pop)
pop_df['genre'] = 'pop'
print(pop_df.shape)
pop_df.head()

(714, 4)


,artist,album,genre,single_count
0,Her Majesty & the Wolves,111,pop,3
1,CocknBullKid,Adulthood,pop,4
2,The Adults,The Adults,pop,0
3,Alexis Jordan,Alexis Jordan,pop,3
4,Amanda Fondell,All This Way,pop,1


### Concat all genres and drop duplicates rows (overlapping genres)

In [48]:
df_list = pd.concat([hiphop_df, rap_df, trap_df, rnb_df, soul_df, funk_df, pop_df])
len(df_list)

1487

In [49]:
df_list = df_list.drop_duplicates(['artist', 'album'])
print(df_list.shape)
df_list.head()

(1094, 4)


,artist,album,genre,single_count
0,Flash Bang Grenada,10 Haters,hiphop,0
1,Aggro Santos,AggroSantos.com,hiphop,3
2,AKA,Altar Ego,hiphop,4
3,Shlohmo,Bad Vibes,hiphop,0
4,Glasses Malone,Beach Cruiser,hiphop,4


### Save to csv file

In [50]:
df_list.to_csv('../data/final_list_debut_album.csv', index=False)